## 2020.AI.MNIST_CNN_LeNet5

In [1]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [3]:
# parameters
learning_rate = 0.9
training_epochs = 50
batch_size = 256

In [4]:
# TorchVision의 MNIST 영상이 28x28로 가공되어 있어, Original 입력 사이즈 32x32로 맞추기 위해
transforms = transforms.Compose([transforms.Resize((32, 32)),
                                 transforms.ToTensor()])

# MNIST dataset
# transform이 mnist_train & mnist_test에 반영되는게 아이라 train_loader & test_loader에서 반영되는 듯
mnist_train = dsets.MNIST(root='MNIST_data/',
                          train=True,
                          transform=transforms,
                          download=True)

mnist_test = dsets.MNIST(root='MNIST_data/',
                         train=False,
                         transform=transforms,
                         download=True)

3.1%

31.0%IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

100.0%


Extracting MNIST_data/MNIST\raw\train-images-idx3-ubyte.gz to MNIST_data/MNIST\raw



102.8%


Extracting MNIST_data/MNIST\raw\train-labels-idx1-ubyte.gz to MNIST_data/MNIST\raw



100.0%


Extracting MNIST_data/MNIST\raw\t10k-images-idx3-ubyte.gz to MNIST_data/MNIST\raw



112.7%

Extracting MNIST_data/MNIST\raw\t10k-labels-idx1-ubyte.gz to MNIST_data/MNIST\raw




c:\python39\lib\site-packages\torchvision\datasets\mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [5]:
# dataset loader
train_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

test_loader = torch.utils.data.DataLoader(dataset=mnist_test,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

In [6]:
class LeNet5(torch.nn.Module):
  def __init__(self):
    super(LeNet5,self).__init__() 

    self.c1 = torch.nn.Conv2d(1,6,kernel_size=5,stride=1,padding=0)
    self.x1 = torch.nn.Sigmoid()
    self.s2 = torch.nn.AvgPool2d(kernel_size=2,stride=2)

    self.c3 = torch.nn.Conv2d(6,16,kernel_size=5,stride=1,padding=0)
    self.x3 = torch.nn.Sigmoid()
    self.s4 = torch.nn.AvgPool2d(kernel_size=2,stride=2) 
    #동일 C5 = F5 
    #self.c5 = torch.nn.Conv2d(16,120,kernel_size=5,stride=1,padding=0)
    self.f5 = torch.nn.Linear(16*5*5,120,bias=True)
    self.x5 = torch.nn.Sigmoid()
    
    self.f6 = torch.nn.Linear(120,84,bias=True)
    self.x6 = torch.nn.Sigmoid()
    
    self.output = torch.nn.Linear(84,10,bias=True)


  def forward(self,x):
    out = self.c1(x)
    out = self.x1(out)
    out = self.s2(out)
    out = self.c3(out)
    out = self.x3(out)
    out = self.s4(out)        
    out = out.view(out.size(0),-1) # batch x 1 x 1 x 120  
    out = self.f5(out)
    out = self.x5(out)
    out = self.f6(out)    
    out = self.x6(out)
    out = self.output(out)  

    return out

In [7]:
class LeNet5_(torch.nn.Module):
  def __init__(self):
    super(LeNet5_,self).__init__() 

    self.c1 = torch.nn.Conv2d(1,6,kernel_size=5,stride=1,padding=0)
    self.x1 = torch.nn.Tanh()
    self.s2 = torch.nn.AvgPool2d(kernel_size=2,stride=2)

    self.c3 = torch.nn.Conv2d(6,16,kernel_size=5,stride=1,padding=0)
    self.x3 = torch.nn.Tanh()
    self.s4 = torch.nn.AvgPool2d(kernel_size=2,stride=2) 

    self.fl = torch.nn.Faltten()

    self.f5 = torch.nn.Linear(16*5*5,120,bias=True)
    self.x5 = torch.nn.Tanh()
    
    self.f6 = torch.nn.Linear(120,84,bias=True)
    self.x6 = torch.nn.Tanh()
    
    self.output = torch.nn.Linear(84,10,bias=True)


  def forward(self,x):
    out = self.c1(x)
    out = self.x1(out)
    out = self.s2(out)
    out = self.c3(out)
    out = self.x3(out)
    out = self.s4(out)        
    out = out.view(out.size(0),-1) # batch x 1 x 1 x 120  
    out = self.f5(out)
    out = self.x5(out)
    out = self.f6(out)    
    out = self.x6(out)
    out = self.output(out)  

    return out

In [8]:
model = LeNet5().to(device)

In [9]:
criterion = torch.nn.CrossEntropyLoss().to(device)   
#optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate) 
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate) 

In [10]:
#train my model
total_batch = len(train_loader)
print('Learning started. It takes sometime.')
for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in train_loader:
        X = X.to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch

    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

print('Learning Finished!')

Learning started. It takes sometime.
[Epoch:    1] cost = 2.31481051
[Epoch:    2] cost = 2.30339408
[Epoch:    3] cost = 2.30240345
[Epoch:    4] cost = 2.30240583
[Epoch:    5] cost = 2.30237007
[Epoch:    6] cost = 2.30200696
[Epoch:    7] cost = 2.30197215
[Epoch:    8] cost = 2.30187225
[Epoch:    9] cost = 2.30174208
[Epoch:   10] cost = 2.30136514
[Epoch:   11] cost = 2.30018353
[Epoch:   12] cost = 2.26611781
[Epoch:   13] cost = 1.78009045
[Epoch:   14] cost = 0.785154462
[Epoch:   15] cost = 0.438564479
[Epoch:   16] cost = 0.289714575
[Epoch:   17] cost = 0.218154922
[Epoch:   18] cost = 0.175280392
[Epoch:   19] cost = 0.150087029
[Epoch:   20] cost = 0.128312007
[Epoch:   21] cost = 0.115893602
[Epoch:   22] cost = 0.103215665
[Epoch:   23] cost = 0.0934744403
[Epoch:   24] cost = 0.0844865367
[Epoch:   25] cost = 0.0779930428
[Epoch:   26] cost = 0.0735645816
[Epoch:   27] cost = 0.0679409578
[Epoch:   28] cost = 0.0635984465
[Epoch:   29] cost = 0.0600517541
[Epoch:   30

In [11]:
accuracy = 0
total_batch = len(test_loader)

with torch.no_grad():
  model.eval()

  for X, Y in test_loader:
    Y = Y.to(device)
    X = X.to(device)


    prediction = model(X)
    correct_prediction = torch.argmax(prediction, 1) == Y
    accuracy += correct_prediction.float().mean()

print('Accuracy:', accuracy.item()/total_batch)



Accuracy: 0.9886818910256411


Accuracy: 0.8358998107910156 -> DNN